<a href="https://colab.research.google.com/github/mpanahi/BERT-crossDomain-sentiment-classification/blob/main/Sentiment_MultiD_ParsBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

dataset_dataframe=pd.read_csv("train_data.csv",usecols=["text","label"])

In [ ]:
dataset_dataframe.head(500)

In [6]:
train_dataframe=dataset_dataframe[0:]

In [ ]:
!pip install transformers

In [ ]:
from  transformers import AutoTokenizer
import torch
tokenizer=AutoTokenizer.from_pretrained("HooshvareLab/bert-fa-zwnj-base")

In [ ]:
from transformers import AutoModelForSequenceClassification

model=AutoModelForSequenceClassification.from_pretrained("HooshvareLab/bert-fa-zwnj-base")

In [10]:
import torch
from torch.utils.data import Dataset, DataLoader

In [11]:
MAX_LEN=512
device="cuda"

In [12]:
class SentimentDataset(Dataset):
  def __init__(self, dataframe):
    self.dataframe=dataframe
  def __len__(self):
    return len(self.dataframe) 
  def __getitem__(self,idx):
    df=self.dataframe.iloc[idx]
    text=[df["text"]]  
    label=[df["label"]] 
    data_t=tokenizer(text,max_length=MAX_LEN,return_tensors="pt",padding='max_length',truncation=True)
    label_t=torch.LongTensor(label)
    return {"input_ids":data_t["input_ids"].squeeze().to(device),"label":label_t.squeeze().to(device),}

In [13]:
train_dataset=SentimentDataset(train_dataframe)

In [ ]:
BATCH_SIZE=16
train_dataloader=DataLoader(train_dataset,batch_size=BATCH_SIZE)
print((train_dataloader.dataset.dataframe)["label"])

In [15]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained("HooshvareLab/bert-fa-zwnj-base").to(device)

In [ ]:
import transformers
from sklearn.metrics import classification_report
import numpy as np
optimizer=transformers.AdamW(model.parameters(),lr=1e-5)

In [18]:
EPOCHS=3
train_dataloader=DataLoader(train_dataset,batch_size=BATCH_SIZE)
print((train_dataloader.dataset.dataframe)["label"])
#EPOCHS=1
for epoch in range(EPOCHS):
 print("\n****\n epoch=",epoch) 
 i=0
 logits_list=[]
 labels_list=[]
 for batch in train_dataloader:
  
  i+=1
  optimizer.zero_grad()
  input_ids=batch["input_ids"]
  
   
  output_model=model(input_ids=batch["input_ids"],labels=batch["label"])
  
  
  loss=output_model.loss
  logits=output_model.logits
  
  logits_list.append(logits.cpu().detach().numpy())
  labels_list.append(batch["label"].cpu().detach().numpy())
  
  loss.backward()
  optimizer.step()
  
 logits_list=np.concatenate(logits_list,axis=0)
 labels_list=np.concatenate(labels_list,axis=0)
 logits_list=np.argmax(logits_list,axis=1)

0        1
1        0
2        1
3        0
4        1
        ..
18177    0
18178    1
18179    0
18180    1
18181    0
Name: label, Length: 18182, dtype: int64

****
 epoch= 0


KeyboardInterrupt: ignored